In [1]:
!python3 -m grpc_tools.protoc -I /opt/proto --python_out=. --pyi_out=. --grpc_python_out=. /opt/proto/game.proto

In [3]:
import game_pb2_grpc
import game_pb2
import grpc
from subprocess import Popen, PIPE
import time
import traceback

In [4]:
def start_server(port, seed):
    p = Popen(['xvfb-run', '--server-num' , str(port), r'./artifical_bike_racing', '--port', str(port), '--seed', str(seed), '--headless', '--save', f'/opt/clients/{port}.json' ], stdout=PIPE, cwd=r'/opt/')
    broken = True
    while broken:
        try:
            channel = grpc.insecure_channel(f'localhost:{port}')
            stub = game_pb2_grpc.MainServiceStub(channel)
            stub.health(game_pb2.Empty())
            broken = False
        except:
            pass
        
    return stub, p.stdout

In [5]:
def kill_server(stub):
    stub.kill(game_pb2.Empty())
    try:
        stub.input(game_pb2.InputRequest(x = 0.0, z = 1.0))
        stub.getState(game_pb2.Empty())
        broken = True
        while broken:
            try:
                stub.health(game_pb2.Empty())
            except:
                broken = False
    except:
        pass

In [6]:
def get_score(stub):
    score = stub.getScore(game_pb2.Empty())
    try:
        stub.input(game_pb2.InputRequest(x = 0.0, z = 1.0))
        stub.getState(game_pb2.Empty())
        broken = True
        while broken:
            try:
                stub.health(game_pb2.Empty())
            except:
                broken = False
    except:
        pass
    return score